In [6]:
import sys
import json
sys.path.append('..')
from src.common import *
from src.analysis.model_performances import *
from src.questions_construction.questions import *
from copy import deepcopy
from helpers import *
import pandas as pd


TO_PRETTY |= {(WITHOUT_RANDOM_SUB, WITHOUT_RAMIFICATIONS): 'Baseline', 
              (WITHOUT_RANDOM_SUB, WITH_RAMIFICATIONS): 'Baseline + R.',
             (WITH_RANDOM_SUB, WITHOUT_RAMIFICATIONS): 'Obfus. Baseline',
             (WITH_RANDOM_SUB, WITH_RAMIFICATIONS): 'Obfus. Baseline + R.',
}

In [32]:
questions_dir = f'{DATA_PATH}/questions_m1'
questions_by_id = gather_questions(questions_dir)
# sanity_checks()

def gather_data_iterator():
    for substitutions in SUBSTITUTION_TYPES:
        for ramifications in RAMIFICATION_TYPES:
            for model_name in ['gemini', 'gpt-4o']:
                for prompt_type in ['few_shot_1']:
                    yield substitutions, ramifications, model_name, prompt_type

ids_file_name = f'dataset_ids.test.pruned' #'small_questions_ids' #
if ids_file_name:
    selected_ids = open_jsonl(f'{DATA_PATH}/{ids_file_name}.jsonl')
    data_all, missing_data = gather_data(questions_by_id, selected_ids=selected_ids, iterator=gather_data_iterator)
    save_main_dir = f'{STATISTICS_PATH}.{ids_file_name}'
else:
    data_all, missing_data = gather_data(questions_by_id)
    save_main_dir = STATISTICS_PATH

save_dir = os.path.join(save_main_dir, 'tables', 'by_fluents')
os.makedirs(save_dir, exist_ok=True)

questions gathered


100%|██████████| 1040/1040 [00:43<00:00, 23.75it/s] 

data is gathered


In [33]:
def to_df_by_category(data_all, model_name, prompt_type,
                      score_type=ACCURACY_SCORE_KEY, 
                      question_category = ALL_QUESTION_CATEGORIES_KEY,
                      domain = ALL_DOMAINS_KEY,
                      answer_type=TRUE_FALSE_ANSWER_TYPE,
                      plan_length=19):

    # index = []
    data_for_df = []    
    for fluent_type in FLUENT_TYPES_LIST:
        # index.append()
        data_columns = {}
        data_columns['fluent type'] = TO_PRETTY[fluent_type]
        for subs in [WITHOUT_RANDOM_SUB, WITH_RANDOM_SUB]:
            for ramifications in [WITHOUT_RAMIFICATIONS, WITH_RAMIFICATIONS]:
                data = filter_multi_selector_modified(data_all, ramifications, model_name, prompt_type, answer_type, subs, plan_length, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])
                stats = TrueFalseStatsCustom(data, plan_length, question_category, ramifications, model_name, prompt_type, domain, subs, score_type=score_type)
                res_obj = stats.compute()
                if res_obj:
                    mean = res_obj['result']
                    sem = None
                    if res_obj['result_other']:
                        sem = res_obj['result_other']['sem']
                    not_corrupted = res_obj['stats']['num_not_corrupted']
                    final_res = (mean, sem, not_corrupted)
                else:
                    final_res = (None, None, None)
                final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
                final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
                # print(final_res)
                data_columns[TO_PRETTY[(subs, ramifications)]] = final_res
        data_for_df.append(data_columns)
    return pd.DataFrame(data_for_df)

# Subs and Ramfications

In [37]:
plan_length = 19
model_name = 'gpt-4o' #'gemini' 
prompt_type = 'few_shot_1'
df = to_df_by_category(data_all, model_name, prompt_type, plan_length=plan_length)
df
        
caption_nl = f'performance of {model_name}, {prompt_type}, pl-{plan_length}'.replace('_', ' ')
save_key = f'{model_name}.{prompt_type}.{plan_length}'

latex_table_all = to_latex_table(df, caption_nl, label=save_key, index=False)
with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
    f.write(latex_table_all)

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:45: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=index, formatters={"name": str.upper}, float_format="{:.2f}".format)
